In [1]:
import numpy as np
import pandas as pd 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
os.path.join(PROJECT_ROOT_DIR)
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
CLEAREDDATA_PATH = os.path.join(PROJECT_ROOT_DIR, "cleareddata")
os.makedirs(IMAGES_PATH, exist_ok=True)
os.makedirs(CLEAREDDATA_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

from pipe_classes import *

./data/scotch_review.csv
./data/nyc_benchmarking_disclosure_2017_consumption_data.xlsx


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
data = pd.read_excel(PROJECT_ROOT_DIR+'/data/nyc_benchmarking_disclosure_2017_consumption_data.xlsx','Information and Metrics')

In [3]:
data = data.replace({'Not Available': np.nan})

In [4]:
data.select_dtypes(exclude=['number']).keys()

Index(['Property Name', 'Parent Property Id', 'Parent Property Name',
       'NYC Borough, Block and Lot (BBL) self-reported',
       'NYC Building Identification Number (BIN)', 'Address 1 (self-reported)',
       'Address 2 (self-reported)', 'Postal Code', 'Street Number',
       'Street Name', 'Borough', 'DOF Gross Floor Area (ft²)',
       'Primary Property Type - Self Selected',
       'List of All Property Use Types at Property',
       'Largest Property Use Type', '2nd Largest Property Use Type',
       '3rd Largest Property Use Type', 'Metered Areas (Energy)',
       'Metered Areas  (Water)', 'Annual Maximum Demand (MM/YYYY)',
       'Water Required?', 'Generation Date',
       'DOF Benchmarking Submission Status'],
      dtype='object')

### First exercise: Lasso and ElasticNet without PCA or collinear features remover and without the sqrt-log feature adder

In [6]:
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

reg_target="ENERGY STAR Score"
spec_cols=['Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_Encoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.
(0, 96)
(24724, 96)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
X = num_pipeline.fit_transform(X)
X_test = num_pipeline.fit_transform(X_test)
X.shape

(17306, 95)

In [9]:
def results_display(reg, X, X_test, y, y_test):
    from sklearn.metrics import mean_squared_error, r2_score
    from timeit import default_timer as timer
    start = timer()
    reg.fit(X, y)
    #train set
    X_pred = reg.predict(X)
    X_rmse = np.sqrt(mean_squared_error(y, X_pred))
    X_R2 = r2_score(y, X_pred)
    #test set
    X_test_pred = reg.predict(X_test)
    X_test_rmse = np.sqrt(mean_squared_error(y_test, X_test_pred))
    X_test_R2 = r2_score(y_test, X_test_pred)
    end = timer()
    print(str(reg))
    print('Train set: Regressor name, RMSE, R2')
    print(reg.__class__.__name__, round(X_rmse,3), round(X_R2,3))
    print('Test set: Regressor name, RMSE, R2')
    print(reg.__class__.__name__, round(X_test_rmse,3), round(X_test_R2,3))
    print('Time consumption [s]: ', round((end - start),3))
    print('---------------------------------------------------------')

In [10]:
from sklearn.linear_model import Lasso
base_reg=Lasso(alpha=1.0, fit_intercept=True, normalize=False, precompute=False, copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')

In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 20 (5×4) combinations of hyperparameters
    {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
    ]

lasso_reg = Lasso(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4335.252245371696, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12842.812687030062, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9591.929673506413, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 883722.00965240

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3644.8074388918467, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10117.76447986206, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3470.494804845657, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9544.87297401

Best estimator: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=500,
      normalize=False, positive=False, precompute=False, random_state=4711,
      selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.001, 'max_iter': 500}


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1976151.1625783397, tolerance: 1770.531869669478
  positive)


In [19]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

18.048306671940594 {'alpha': 1.0, 'max_iter': 200}
18.048021913851716 {'alpha': 1.0, 'max_iter': 500}
18.048021913851716 {'alpha': 1.0, 'max_iter': 1000}
18.048021913851716 {'alpha': 1.0, 'max_iter': 2000}
17.395068921873893 {'alpha': 0.1, 'max_iter': 200}
17.39467804790433 {'alpha': 0.1, 'max_iter': 500}
17.39467804790433 {'alpha': 0.1, 'max_iter': 1000}
17.39467804790433 {'alpha': 0.1, 'max_iter': 2000}
17.34763692241368 {'alpha': 0.01, 'max_iter': 200}
17.344769985240273 {'alpha': 0.01, 'max_iter': 500}
17.34383517386449 {'alpha': 0.01, 'max_iter': 1000}
17.34376158381778 {'alpha': 0.01, 'max_iter': 2000}
17.3480465376241 {'alpha': 0.001, 'max_iter': 200}
17.343249252824993 {'alpha': 0.001, 'max_iter': 500}
17.343525111117938 {'alpha': 0.001, 'max_iter': 1000}
17.343662528938527 {'alpha': 0.001, 'max_iter': 2000}
17.34841031300705 {'alpha': 0.0001, 'max_iter': 200}
17.343583084248323 {'alpha': 0.0001, 'max_iter': 500}
17.34381929011344 {'alpha': 0.0001, 'max_iter': 1000}
17.34401733

In [20]:
results_display(grid_best_est, X, X_test, y, y_test)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=500,
      normalize=False, positive=False, precompute=False, random_state=4711,
      selection='cyclic', tol=0.0001, warm_start=False)
Train set: Regressor name, RMSE, R2
Lasso 17.276 0.708
Test set: Regressor name, RMSE, R2
Lasso 17.764 0.689
Time consumption [s]:  0.563
---------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1976151.1625783397, tolerance: 1770.531869669478
  positive)


In [21]:
from sklearn.linear_model import ElasticNet
base_reg2=ElasticNet(alpha=1.0, l1_ratio=0.5, fit_intercept=True, normalize=False, precompute=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')

In [22]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 32 (3×4x3) combinations of hyperparameters
    {'alpha': [0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000], 'l1_ratio': [0.3, 0.5, 0.7]},
    ]

elnet_reg = ElasticNet(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 32*5=160 rounds of training 
grid_search = GridSearchCV(elnet_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 774654.2256771005, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 845625.8253314912, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 866664.3770075499, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 892756.079639

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23869.573617741466, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18407.10618240107, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 30453.64335955819, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1991148.3598248

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1749494.8422003696, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 408823.4299383438, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 671123.0977854638, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 455742.2065163

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2058360.5448086325, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2059175.5356780682, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2071970.994113923, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2060930.9884

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1489482.5321509917, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1918894.9246283195, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1474141.072076357, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1849864.967613

Best estimator: ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.7,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.001, 'l1_ratio': 0.7, 'max_iter': 1000}


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 395482.7584486874, tolerance: 1770.531869669478
  positive)


In [23]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

17.369316494345288 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
17.370618900411234 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 500}
17.370618900411234 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 1000}
17.370618900411234 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 2000}
17.36173730710027 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 200}
17.363068240108205 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 500}
17.363068240108205 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 1000}
17.363068240108205 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 2000}
17.355137111767476 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 200}
17.355339712066563 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 500}
17.355339712066563 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 1000}
17.355339712066563 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 2000}
17.34805495454793 {'alpha': 0.001, 'l1_ratio': 0.3, 'max_iter': 200}
17.34396419931581 {'alpha': 0.001, 'l1_ratio': 0.3, 'max_iter': 500}
17.34358444342495 {'alpha': 0

In [24]:
results_display(grid_best_est, X, X_test, y, y_test)

ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.7,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Train set: Regressor name, RMSE, R2
ElasticNet 17.276 0.708
Test set: Regressor name, RMSE, R2
ElasticNet 17.755 0.689
Time consumption [s]:  1.536
---------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 395482.7584486874, tolerance: 1770.531869669478
  positive)


first observations:
- both Lasso and ElasticNet perform quite good and are fast
- both Lasso and ElasticNet show R2-scores of ~71% on training set and ~69% on test set

### Second exercise: Lasso and ElasticNet with the sqrt-log feature adder

In [25]:
spec_cols=['Water Use (All Water Sources) (kgal)','log_Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_AdderEncoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/notebooks/ML/pipe_classes.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
/notebooks/ML/pipe_classes.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['log_' + col] = np.log(numeric_subset[col])
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856:

(0, 143)
(24724, 143)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
X = num_pipeline.fit_transform(X)
X.shape

(17306, 142)

In [27]:
X_test = num_pipeline.fit_transform(X_test)

In [28]:
param_grid = [
    # try 20 (5×4) combinations of hyperparameters
    {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
    ]

lasso_reg = Lasso(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5629.113326133229, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2354.624914081767, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2370.471618469339, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3983.959935400

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1641839.248712887, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1663842.742752234, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1564957.6736185714, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1581490.8946964

Best estimator: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=2000,
      normalize=False, positive=False, precompute=False, random_state=4711,
      selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.001, 'max_iter': 2000}


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1777454.5092600458, tolerance: 1770.531869669478
  positive)


In [29]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

17.174082230798987 {'alpha': 1.0, 'max_iter': 200}
17.174082230798987 {'alpha': 1.0, 'max_iter': 500}
17.174082230798987 {'alpha': 1.0, 'max_iter': 1000}
17.174082230798987 {'alpha': 1.0, 'max_iter': 2000}
16.3449737989269 {'alpha': 0.1, 'max_iter': 200}
16.344354728069458 {'alpha': 0.1, 'max_iter': 500}
16.344354728069458 {'alpha': 0.1, 'max_iter': 1000}
16.344354728069458 {'alpha': 0.1, 'max_iter': 2000}
16.186167133341154 {'alpha': 0.01, 'max_iter': 200}
16.162546952918923 {'alpha': 0.01, 'max_iter': 500}
16.149887610036345 {'alpha': 0.01, 'max_iter': 1000}
16.142958763776946 {'alpha': 0.01, 'max_iter': 2000}
16.211427471780638 {'alpha': 0.001, 'max_iter': 200}
16.17310898935317 {'alpha': 0.001, 'max_iter': 500}
16.14408921322154 {'alpha': 0.001, 'max_iter': 1000}
16.120537264902616 {'alpha': 0.001, 'max_iter': 2000}
16.21421102709635 {'alpha': 0.0001, 'max_iter': 200}
16.17739149116222 {'alpha': 0.0001, 'max_iter': 500}
16.14674425991365 {'alpha': 0.0001, 'max_iter': 1000}
16.12142

In [30]:
results_display(grid_best_est, X, X_test, y, y_test)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=2000,
      normalize=False, positive=False, precompute=False, random_state=4711,
      selection='cyclic', tol=0.0001, warm_start=False)
Train set: Regressor name, RMSE, R2
Lasso 15.995 0.75
Test set: Regressor name, RMSE, R2
Lasso 16.622 0.728
Time consumption [s]:  4.355
---------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1777454.5092600458, tolerance: 1770.531869669478
  positive)


In [31]:
param_grid = [
    # try 32 (3×4x3) combinations of hyperparameters
    {'alpha': [0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000], 'l1_ratio': [0.3, 0.5, 0.7]},
    ]

elnet_reg = ElasticNet(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(elnet_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1447741.8023507602, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1424984.0537825897, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1436283.4968841337, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1469339.04

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1179068.951118772, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1265923.5241741075, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1275437.7844998748, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18808.45149667

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1404025.3647917027, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1748056.3052007481, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1752942.386655569, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1737718.46770

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1604138.8644401655, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1582296.6349639783, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1586737.1435903814, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1635655.883

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1797224.3341096293, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1805445.2179992395, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1762480.866739488, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1768661.308176

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1733129.7794682242, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1738606.225281436, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1728705.0758278384, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1751903.935

Best estimator: ElasticNet(alpha=0.0001, copy_X=True, fit_intercept=True, l1_ratio=0.7,
           max_iter=2000, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.0001, 'l1_ratio': 0.7, 'max_iter': 2000}


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2182030.0941083287, tolerance: 1770.531869669478
  positive)


In [32]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

16.24344555475759 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
16.255854980329683 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 500}
16.254209353835908 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 1000}
16.25462429459646 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 2000}
16.223528176189422 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 200}
16.231150029224548 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 500}
16.23237258498199 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 1000}
16.23222640845116 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 2000}
16.204200971042653 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 200}
16.204084599718662 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 500}
16.207482226150994 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 1000}
16.20753030605079 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 2000}
16.20860371821683 {'alpha': 0.001, 'l1_ratio': 0.3, 'max_iter': 200}
16.172744250107613 {'alpha': 0.001, 'l1_ratio': 0.3, 'max_iter': 500}
16.153781490670983 {'alpha': 0.0

In [33]:
results_display(grid_best_est, X, X_test, y, y_test)

ElasticNet(alpha=0.0001, copy_X=True, fit_intercept=True, l1_ratio=0.7,
           max_iter=2000, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Train set: Regressor name, RMSE, R2
ElasticNet 15.992 0.75
Test set: Regressor name, RMSE, R2
ElasticNet 16.641 0.727
Time consumption [s]:  4.052
---------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2182030.0941083287, tolerance: 1770.531869669478
  positive)


observations:
- both Lasso and ElasticNet show performance improvement with log and sqrt features added
- both Lasso and ElasticNet show R2-scores of ~75% (base: 71%) on training set and ~73% (base: 69%) on test set